# 딥러닝 안쓰고 주가 예측하기 part1

본 tutorial에선 deep learning을 사용하지 않고 주가를 예측하는 방법에 대하여 다룹니다.

<다룰 내용>
- Pearson correlation coefficient

In [ ]:
# 이번 tutorial에서 사용할 파이썬 라이브러리를 불러옵니다.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance
import pandas_datareader as pdr
from datetime import datetime, timedelta, timezone

%matplotlib inline

## Pearson correlation coefficient
Pearson correlation coefficient(피어슨 상관계수)는 두 변수 사이의 상관관계를 수치적으로 계산할 때 사용합니다.

상관계수란 두 벡터 사이의 관계를 수치적으로 나타낸 것으로, 예컨대 $x$가 증가할 때 $y$가 증가한다면 상관계수가 양수이고 그 반대의 경우엔 음수입니다.
상관계수 $\rho$는 $[-1, 1]$ 구간의 값을 갖습니다. 두 벡터사이의 양의 상관관계가 강할수록 1에 수렴하고, 정반대로 움직이는 경우 -1에 수렴합니다.

In [ ]:
X = np.linspace(0, 10, 1000)

# 간단한 두 벡터를 만들어봅시다.
Y1 = X + np.random.randn(X.shape[0])
Y2 = X + np.random.randn(X.shape[0]) + 0.5

In [ ]:
# 두 벡터의 상관관계를 눈으로 확인해봅시다.
plt.scatter(X, Y1, s=2)
plt.scatter(X, Y2, s=2)

In [ ]:
# Y1이 증가하면 Y2도 증가하는, 양의 상관관계를 가지는 것을 눈으로 확인할 수 있습니다.
# Pearson correlation을 계산하여 어떤 값을 갖는지 확인해봅시다.
print('pearson coef:', np.corrcoef(x=Y1, y=Y2))

In [ ]:
# 이번에는 음의 상관관계를 갖는 두 벡터를 만들어봅시다.
Y1 = X + np.random.randn(X.shape[0])
Y2 = 10 - X - np.random.randn(X.shape[0])

In [ ]:
# 두 벡터의 상관관계를 눈으로 확인해봅시다.
plt.scatter(X, Y1, s=2)
plt.scatter(X, Y2, s=2)

In [ ]:
# Y1이 증가하면 Y2는 감소하는, 음의 상관관계를 가지는 것을 눈으로 확인할 수 있습니다.
# Pearson correlation을 계산하여 어떤 값을 갖는지 확인해봅시다.
print('pearson coef:', np.corrcoef(x=Y1, y=Y2))

## 피어슨 상관계수를 이용한 주가 예측

피어슨 상관계수를 이용해 두 변수의 움직임의 관계를 수치적으로 확인할 수 있음을 보았습니다.
이제 위 내용을 바탕으로 주가 예측을 시도해봅시다.

In [ ]:
# 지금까지의 모든 나스닥 지수(^IXIC)를 가져옵니다.
now = datetime.now(timezone(timedelta(hours=-4)))
df = pdr.get_data_yahoo('^IXIC', start='1971-02-05', end=now.strftime('%Y-%m-%d'))
df = df[['Close']]

# 전일 종가대비 금일 종가의 수익률(return)을 계산합니다.
df['Return'] = df['Close'].pct_change()
df = df.iloc[1:]  # 첫 번째 데이터는 전날 종가가 없으므로 NaN이 나옵니다. 따라서 제거하도록 합니다.

df

cf) $Return(t) = \frac{S_t - S_{t-1}}{S_{t-1}}$

In [ ]:
# 나스닥 지수의 종가 그래프를 그려봅니다.
plt.figure(figsize=(12, 4))
sns.lineplot(data=df, x='Date', y='Close')

In [ ]:
# 이번엔 나스닥 지수의 일일 수익률 그래프를 그려봅시다.
plt.figure(figsize=(12, 4))
sns.lineplot(data=df, x='Date', y='Return')

단순히 지수만 보았을 때보단 수익률을 보았을 때 유사한 패턴이 많이 나올 것 같습니다.

최근 수익률 패턴과 유사한 과거 차트를 찾아 과거엔 주가가 상승했는지, 하락했는지를 보고 가격 예측을 하는 모델을 만들어봅시다.

In [ ]:
# 20 거래일만큼 지수 데이터를 잘라내어 데이터셋을 만들어봅시다.
WINDOW_SIZE = 20
DATASET_SIZE = len(df) - 2 * WINDOW_SIZE + 1

Xs = np.zeros((DATASET_SIZE, WINDOW_SIZE))
Ys = np.zeros((DATASET_SIZE, WINDOW_SIZE))

for i in range(DATASET_SIZE):
    Xs[i] = df.iloc[i:i + WINDOW_SIZE]['Return'].to_numpy()
    Ys[i] = df.iloc[i + WINDOW_SIZE:i + 2 * WINDOW_SIZE]['Return'].to_numpy()

In [ ]:
# Xs끼리의 피어슨 상관계수를 구해봅시다.
# 이때 Xs[-1]은 가장 최근 20거래일의 수익률 지표입니다.
coef = np.corrcoef(Xs)[-1, :-1]
coef

In [ ]:
# 가장 비슷한 수익률 추이를 보이는 날과, 가장 반대되는 수익률 추이를 보이는 날을 찾아봅시다.
max_id = np.argmax(coef)
min_id = np.argmin(coef)

print('maximum coef:', coef[max_id])
print('minimum coef:', coef[min_id])

In [ ]:
# 실제로 그러한지 그래프를 그려봅시다.
x_axis = np.arange(0, WINDOW_SIZE)

plt.figure(figsize=(12, 4))
plt.plot(x_axis, Xs[max_id], label='maximum coef')
plt.plot(x_axis, Xs[-1], label='latest')
plt.plot(x_axis, Xs[min_id], label='minimum coef')
plt.legend(loc='best')

위 사례를 통해 주가 데이터에서 피어슨 상관계수를 이용해 유사하거나 완전히 반대되는 시기를 찾을 수 있음을 확인했습니다.
또한, 수치적으로 얼마만큼 유사한지도 구할 수 있었습니다.

이제, 수익률 추이가 비슷하면 그 후에 주가의 향방도 유사할거란 가정을 하고 모델을 만들어봅시다.
위에서 얻은 정보를 이용하는 방법은 다양하게 존재합니다.
이번 tutorial에선 가장 비슷한 상위 $N$개를 평균내어 예측하는 것으로 해보겠습니다.

In [ ]:
# 가장 비슷한 상위 30개를 구해 weighted average을 내어봅시다.
K = 30
top_k = np.argpartition(coef, -K)[-K:]

predict = np.sum(np.multiply(Ys[top_k], coef[top_k].reshape((-1, 1))), axis=0) / np.sum(coef[top_k])
predict

In [ ]:
# 예상 수익률을 바탕으로 예상 주가를 계산해봅시다.
last_price = df['Close'].iloc[-1]
start_price = last_price
predict_prices = np.zeros(WINDOW_SIZE)
for i in range(WINDOW_SIZE):
    last_price = last_price * (1 + predict[i])
    predict_prices[i] = last_price
    
print('20 거래일 후 예상 지수: {:.2f}'.format(last_price))
print('20 거래일 후 예상 수익률: {:.2f}%'.format((last_price - start_price) / start_price * 100))

In [ ]:
# 예상 수익률을 바탕으로 그림을 그려봅시다.
# 겸사겸사 top-k에 해당하는 주가 변화도 그려봅시다.
plt.figure(figsize=(12, 4))
plt.plot(x_axis, predict_prices / start_price, linewidth=5)

for i in top_k:
    prices = np.zeros(WINDOW_SIZE)
    prices[0] = 1 + Ys[i, 0]
    for j in range(1, WINDOW_SIZE):
        prices[j] = prices[j - 1] * (1 + Ys[i, j])
        
    plt.plot(x_axis, prices, alpha=0.3, linewidth=1)